# 3. Generating 1sqkm squares

The hexagons we generated on the previous steps might be useful for data analysis and for plotting maps, but we also want to highlight specific places. For those, it would be weird to show an arbitrary shape such as this. To overcome this,  we will generate squares that are associated with each of the hexagons, albeit with slightly different tree coverage values and some degree of overlap.

This way, when plotting those satellite images, we can give an idea of how a 25% tree-coverage area looks like, while using a shape that won't confuse readers as much.

Please notice that this can't be used for any kind of analysis, as the squares will overlap significantly. It's useful for visualization purposes only.

In [6]:
# Importing packages
from datetime import datetime
import ee
import geemap
import geopandas as gpd

In [96]:
def main():
    
    # Initializing Earth Engine
    ee.Initialize()
    
    # A timestamp
    now = datetime.now().strftime("%d-%m-%y-%H:%M").replace(":","_")
    
    # Loads the same Images as in the previous notebook, applying the same
    # masks and calculations.
    tree_coverage_1m = ee.ImageCollection('projects/meta-forest-monitoring-okw37/assets/CanopyHeight').mosaic()
    tree_coverage_10m = ee.Image('users/nlang/ETH_GlobalCanopyHeight_2020_10m_v1')
    tree_coverage_1m = tree_coverage_1m.updateMask(tree_coverage_1m.gte(1))
    tree_coverage_10m = tree_coverage_10m.updateMask(tree_coverage_10m.gte(1))
    tree_coverage_mask = tree_coverage_10m.gte(1)
    tree_coverage = tree_coverage_1m.updateMask(tree_coverage_mask)
    population = ee.Image('JRC/GHSL/P2023A/GHS_POP/2020')
    
    # Loads the hexagons
    hexagons = ee.FeatureCollection("projects/dw-city-tree-coverage/assets/city_hexagons")
    
    # Creates the pixel area object
    pixel_area = ee.Image.pixelArea()

    # Maps over the hexagons creating a square of 1km around its centroid
    # by first creating a 500m radius buffer and then getting the bounding box.
    # Since its projected over the globe, it may vary slightly with latitude.
    # Also computes the population living there and the share of tree coverage,
    # in a similar fashion to what was done in the previous notebook.
    def make_box(feature):
        
        # Creates the polygon
        centroid = feature.geometry().centroid() # Gets the centroid of the hexagon
        buffer = centroid.buffer(500) # Circle with 500m radius
        box = buffer.bounds() # Bounding box containing the circle, with 1km of side
        
        # Gets the area of the box
        ft_area = box.area(.1) # Small error margin
        
        # Also saves the lat and lon of the centroid
        lon = centroid.coordinates().get(0)
        lat = centroid.coordinates().get(1)
        
        # Computes the tree cover area using the pixel area object
        tree_area_image = tree_coverage.gte(1).multiply(pixel_area)
        tree_area = tree_area_image.reduceRegion(
            reducer = ee.Reducer.sum(),
            geometry = box, # The geometry is now the BOUNDING BOX OF THE CIRCLE
            scale = 1,
            maxPixels = 1e12
        ).get('cover_code')
        
        # Similarly, compute the population on the box
        pop_ft = population.reduceRegion(
            reducer = ee.Reducer.sum(),
            geometry = feature.geometry(), # The box is the area of interest now
            scale = 100,
            maxPixels = 1e12,
        ).get('population_count')
                
        # And returns the feature, this time making sure to save the new geometry as well
        return feature.setGeometry(box)\
                      .set({
                        'lon': lon,
                        'lat': lat,
                        'ft_area': ft_area,
                        'tree_area': tree_area,
                        'tree_pct': ee.Number(tree_area).divide(ee.Number(ft_area)), # Uses earth engine Number to divide server side
                        'pop_ft': pop_ft
                    })

        
    # Runs the computation
    boxes = hexagons.map(lambda feature: make_box(feature))
    
    # And exports the boxes to a separate directory within Google Drive.
    # Since we have geo information, we are using geojson as the export format.
    geemap.ee_export_vector_to_drive(
        collection = boxes,
        folder =  'DWTreeCoverAnalysis',
        description = f'TreeCoverAnalysis-Boxes-{now}',
        fileFormat = 'SHP',
        selectors = ['city_id', 'hexagon_n',
                     'lon', 'lat', 
                     'tree_area', 'ft_area', 'tree_pct', 'pop_ft']
    )

In [97]:
if __name__ == "__main__":
    main()

{'type': 'Feature', 'geometry': {'geodesic': False, 'type': 'Polygon', 'coordinates': [[[-17.50569479754689, 14.7503823025731], [-17.496450166823735, 14.7503823025731], [-17.496450166823735, 14.759379439463675], [-17.50569479754689, 14.759379439463675], [-17.50569479754689, 14.7503823025731]]]}, 'id': '00000000000000000000', 'properties': {'area': 0.723621187206863, 'city_id': 1452, 'ft_area': 1001255.4926034017, 'hexagon_n': 0, 'lat': 14.7548800064879, 'lon': -17.501079817885895, 'pop_ft': 2752.5696444709038, 'tree_area': 1089.6351030756446, 'tree_pct': 0.001088268789659714}}
Exporting TreeCoverAnalysis-Boxes-26-08-24-17_26... Please check the Task Manager from the JavaScript Code Editor.
